<a href="https://colab.research.google.com/github/eobianyor/Project3/blob/main/ColabModel4IdahoFires_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [4]:
# IMPORT DEPENDENCIES
# FOR DATA
import pandas as pd
# import os
import csv
import math
import numpy as np
from numpy import logspace
import time
import datetime
# import requests
# import datefinder

# # FOR SQL LITE
# from sqlalchemy import create_engine
# from datetime import date

# # FOR PLOTTING
import matplotlib.pyplot as plt
from matplotlib import style
style.use("fivethirtyeight")
from matplotlib import rcParams
rcParams['figure.figsize'] = 10, 8

# FOR MODELING
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

# FOR KNN AND OTHER MODELS
from scipy.optimize import curve_fit
# from splinter import Browser
# from bs4 import BeautifulSoup as BS
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.datasets import make_blobs
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

# # FOR TF MODELS
# import keras.models
# import keras.layers
# import keras.utils
# from keras.models import Sequential
# from keras.layers import Dense, Dropout

In [5]:
# IMPORT LIBRARIES FOR COLAB
# =========================================================
#               Python Libraries for Colab
# =========================================================
import sys
# ---------------------------------------------------------
from google.colab import drive

# =========================================================
#               Mount Google Drive to Colab
# =========================================================
drive.mount('/content/gdrive')


Mounted at /content/gdrive


IMPORT DATA

In [7]:
# File path for the CSV Files
# idahoFireWeatherDrought = os.path.join("/content/gdrive/My Drive/Colab Notebooks/IdahoFirePorject/fires_Idaho_2000_2015_drought_weather_Plus.csv")
# idahoFireWeatherDrought = "/content/gdrive/My Drive/Colab Notebooks/IdahoFireProject/fires_Idaho_2000_2015_drought_weather_Plus.csv"
idahoFireWeatherDrought = "/content/gdrive/My Drive/Colab Notebooks/IdahoFireProject/EXAMPLE_wthr_fire_strmflw_lghtnng_merged_df.csv"

# Open the CSV Files, Convert to a Dataframe, and Save as a Variable
idaho_Fire_Weather_Drought_df = pd.read_csv(idahoFireWeatherDrought)
# fires_Idaho_df = pd.read_csv(idahoFires, dtype={"LOCAL_INCIDENT_ID": 'string', "FIRE_NAME": 'string'})

In [8]:
idahoCounties = ['Ada', 'Adams', 'Bannock', 'Bear Lake', 'Benewah', 'Bingham', 'Blaine', 'Boise', 'Bonner', 'Bonneville', 'Boundary','Butte', 
 'Camas', 'Canyon', 'Caribou', 'Cassia', 'Clark', 'Clearwater', 'Custer', 'Elmore', 'Franklin', 'Fremont', 'Gem', 'Gooding', 
 'Idaho', 'Jefferson', 'Jerome' 'Kootenai', 'Latah', 'Lemhi', 'Lewis', 'Lincoln', 'Madison', 'Minidoka','Nez Perce', 
 'Oneida', 'Owyhee', 'Payette', 'Power', 'Shoshone', 'Teton', 'Twin Falls', 'Valley', 'Washington']
        

notIdahoCounties = ['Baker', 'Beaverhead', 'Box Elder', 'Elko', 'Gallatin', 'Malheur', 'Mineral', 'Missoula', 'Ravalli', 'Sanders']

DATA CLEAN

In [ ]:
# # CONVERT DATE TO DATETIME FORMAT
# idaho_Fire_Weather_Drought_df['DISCOVERY_DATE_CONVERTED'] = pd.to_datetime(idaho_Fire_Weather_Drought_df['DISCOVERY_DATE_CONVERTED'])
# idaho_Fire_Weather_Drought_df['CONT_DATE_CONVERTED'] = pd.to_datetime(idaho_Fire_Weather_Drought_df['CONT_DATE_CONVERTED'])

# # CREATE COLUMNS WE NEED
# idaho_Fire_Weather_Drought_df['FIRE_DAYS'] = (((idaho_Fire_Weather_Drought_df['CONT_DOY']) + 1) - idaho_Fire_Weather_Drought_df['DISCOVERY_DOY'])
# idaho_Fire_Weather_Drought_df['COUNTY_NAME'] = (idaho_Fire_Weather_Drought_df['FIPS_NAME'])
# # Create month column
# idaho_Fire_Weather_Drought_df['DISCOVERY_MONTH_CONVERTED']=idaho_Fire_Weather_Drought_df['DISCOVERY_DATE_CONVERTED'].apply(lambda x: int(x.strftime('%m')))

# # FILTER ROWS FOR FIRES IN COUNTIES OUTSIDE IDAHO
# idaho_Fire_Weather_Drought_df = idaho_Fire_Weather_Drought_df[idaho_Fire_Weather_Drought_df['NAME'].isin(idahoCounties)]

# # View the Data in the Dataframe
# # print(idaho_Fire_Weather_df.keys())
# idaho_Fire_Weather_Drought_df

In [ ]:
# # Get Avg for values
# Day1_prcp = idaho_Fire_Weather_Drought_df['DAY_PRCP_1'].mean()
# Day2_prcp = idaho_Fire_Weather_Drought_df['DAY_PRCP_2'].mean()
# Day3_prcp = idaho_Fire_Weather_Drought_df['DAY_PRCP_3'].mean()
# Day4_prcp = idaho_Fire_Weather_Drought_df['DAY_PRCP_4'].mean()
# Day1_temp = idaho_Fire_Weather_Drought_df['DAY_AVG_TEMP_1'].mean()
# Day2_temp = idaho_Fire_Weather_Drought_df['DAY_AVG_TEMP_2'].mean()
# Day3_temp = idaho_Fire_Weather_Drought_df['DAY_AVG_TEMP_3'].mean()
# Day4_temp = idaho_Fire_Weather_Drought_df['DAY_AVG_TEMP_4'].mean()

# # Use Avg values to fill any null values
# idaho_Fire_Weather_Drought_df['DAY_PRCP_1'] = idaho_Fire_Weather_Drought_df['DAY_PRCP_1'].fillna(Day1_prcp)
# idaho_Fire_Weather_Drought_df['DAY_PRCP_2'] = idaho_Fire_Weather_Drought_df['DAY_PRCP_2'].fillna(Day2_prcp)
# idaho_Fire_Weather_Drought_df['DAY_PRCP_3'] = idaho_Fire_Weather_Drought_df['DAY_PRCP_3'].fillna(Day3_prcp)
# idaho_Fire_Weather_Drought_df['DAY_PRCP_4'] = idaho_Fire_Weather_Drought_df['DAY_PRCP_4'].fillna(Day4_prcp)
# idaho_Fire_Weather_Drought_df['DAY_AVG_TEMP_1'] = idaho_Fire_Weather_Drought_df['DAY_AVG_TEMP_1'].fillna(Day1_temp)
# idaho_Fire_Weather_Drought_df['DAY_AVG_TEMP_2'] = idaho_Fire_Weather_Drought_df['DAY_AVG_TEMP_2'].fillna(Day2_temp)
# idaho_Fire_Weather_Drought_df['DAY_AVG_TEMP_3'] = idaho_Fire_Weather_Drought_df['DAY_AVG_TEMP_3'].fillna(Day3_temp)
# idaho_Fire_Weather_Drought_df['DAY_AVG_TEMP_4'] = idaho_Fire_Weather_Drought_df['DAY_AVG_TEMP_4'].fillna(Day4_temp)

In [ ]:
# ### CORRELATION TESTS

# Corr_df = idaho_Fire_Weather_Drought_df[['FIRE_SIZE_CLASS', 'AVE_SIZE12', 'CROP_ACR12', 'NAME', 'None', 'D0','D1', 'D2', 'D3', 'D4', 
#                                    'DAY_PRCP_1', 'DAY_PRCP_2', 'DAY_PRCP_3', 'DAY_PRCP_4', 'DAY_AVG_TEMP_1', 'DAY_AVG_TEMP_2', 'DAY_AVG_TEMP_3', 'DAY_AVG_TEMP_4']]
# Corr_df['FIRE_SIZE_CLASS']= Corr_df['FIRE_SIZE_CLASS'].astype('category').cat.codes

# Corr_df.corr()

In [9]:
idaho_Fire_Weather_Drought_df

,Unnamed: 0,avg_temp,prcp,humidity,dew_point,strms_blw_extndd_wghtd_avg,number_of_strikes,fire_true_no_fire_false
0,0,12.021967,16.325000,53.957773,0.712846,5,9,False
1,1,9.371115,27.066667,60.448122,0.536316,9,4,False
2,2,11.612043,19.925000,53.915149,0.219563,5,8,True
3,3,11.312903,29.883333,58.198352,1.722565,6,10,False
4,4,11.190225,30.033333,56.924064,1.263816,7,2,True
5,5,11.454038,23.525000,56.201823,1.587403,8,10,True
6,6,12.096963,35.416667,59.353768,2.920884,8,4,True
7,7,11.590760,18.208333,52.477421,0.254361,10,7,True
8,8,11.568799,25.550000,57.386226,1.446670,25,6,True
9,9,11.670478,18.150000,55.479528,1.025720,1,4,True


EXTRACT X AND y FOR MODEL

In [10]:
# Create X Input by keeping columns from df
X = idaho_Fire_Weather_Drought_df[['avg_temp', 'prcp', 'humidity', 'dew_point', 'strms_blw_extndd_wghtd_avg', 'number_of_strikes']]

# X by drop
# X = idaho_Fire_Weather_df.drop(['FIRE_SIZE_CLASS', 'DISCOVERY_DATE_CONVERTED'], axis=1)

# Get dummies for names (if using the county 'NAME' column)
# X = pd.get_dummies(X, columns=['NAME'])

print(X.keys())
X = X.values.reshape(-1, 6)

# X = X.to_numpy()

print(X.shape)
type(X)
X

Index(['avg_temp', 'prcp', 'humidity', 'dew_point',
       'strms_blw_extndd_wghtd_avg', 'number_of_strikes'],
      dtype='object')
(24, 6)


array([[ 1.20219673e+01,  1.63250000e+01,  5.39577732e+01,
         7.12845832e-01,  5.00000000e+00,  9.00000000e+00],
       [ 9.37111482e+00,  2.70666667e+01,  6.04481224e+01,
         5.36315593e-01,  9.00000000e+00,  4.00000000e+00],
       [ 1.16120429e+01,  1.99250000e+01,  5.39151495e+01,
         2.19562639e-01,  5.00000000e+00,  8.00000000e+00],
       [ 1.13129032e+01,  2.98833333e+01,  5.81983516e+01,
         1.72256502e+00,  6.00000000e+00,  1.00000000e+01],
       [ 1.11902249e+01,  3.00333333e+01,  5.69240640e+01,
         1.26381558e+00,  7.00000000e+00,  2.00000000e+00],
       [ 1.14540382e+01,  2.35250000e+01,  5.62018227e+01,
         1.58740308e+00,  8.00000000e+00,  1.00000000e+01],
       [ 1.20969630e+01,  3.54166667e+01,  5.93537682e+01,
         2.92088443e+00,  8.00000000e+00,  4.00000000e+00],
       [ 1.15907597e+01,  1.82083333e+01,  5.24774211e+01,
         2.54361230e-01,  1.00000000e+01,  7.00000000e+00],
       [ 1.15687990e+01,  2.55500000e+01,  5.738

In [11]:
### CREATE y VALUES
y = idaho_Fire_Weather_Drought_df[['fire_true_no_fire_false']]

# NO OF OUTPUTS (FOR ML MODELING)
NoOfOutputs = 2

# y = y.values.reshape(-1, 2)

# print(y.shape)
# # type(y)
# y

# LABEL ENCODE Y
# Import required module
from sklearn.preprocessing import LabelEncoder

# LabelEncoder
le = LabelEncoder()

# Create an object of the label encoder class
labelencoder = LabelEncoder()

# apply "le.fit_transform"
old_y = y.apply(le.fit_transform)
y = old_y

# # Change the shape of y v1
new_y = np.array(old_y)
y = new_y.reshape(-1, 1) 

# View output
print(f"Shape of y is {y.shape}")
print(f"Type of y is {type(y)}")
y

Shape of y is (24, 1)
Type of y is <class 'numpy.ndarray'>


array([[0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1]])

FORMAT MODEL INPUTS

In [12]:
# Split data into train and test groups
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [13]:
# # Scale your data
# from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)
# # y_scaler = StandardScaler().fit(y_train)

# # Create variables to hold the scaled train & test data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
# # y_train_scaled = y_scaler.transform(y_train)
# # y_test_scaled = y_scaler.transform(y_test)

In [14]:
### Encode the categorical target variable to the necessary format for the model
from keras.utils import to_categorical

# One-hot encoding
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [15]:
# X Inputs
print(X_train.shape)
print(X_train_scaled.shape)
print(X_test.shape)
print(X_test_scaled.shape)

# y Inputs
print(y_train_categorical.shape)
print(y_test_categorical.shape)

(18, 6)
(18, 6)
(6, 6)
(6, 6)
(18, 2)
(6, 2)


MODELS

In [16]:
# Create the GridSearchCV model
model1 = SVC(kernel="rbf")
model1

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [51]:
# Create the GridSearchCV parameters

# 'gamma': [0.0000001, 0.0000005, 0.0000001, 0.0000005, 0.000001, 0.000005, 0.000001]

param_grid = {'C': [0.01],
              'gamma': [0.00000001],
              'degree': [-0.9],
              'tol': [0.000000005]}

cv = KFold(n_splits=5, shuffle=False, random_state=None)
gridModel = GridSearchCV(model1, param_grid, verbose=3, scoring='accuracy', cv=cv)

In [52]:
# Train the model with GridSearch
gridModel.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] C=0.01, degree=-0.9, gamma=1e-08, tol=5e-09 .....................
[CV]  C=0.01, degree=-0.9, gamma=1e-08, tol=5e-09, score=0.750, total=   0.0s
[CV] C=0.01, degree=-0.9, gamma=1e-08, tol=5e-09 .....................
[CV]  C=0.01, degree=-0.9, gamma=1e-08, tol=5e-09, score=0.750, total=   0.0s
[CV] C=0.01, degree=-0.9, gamma=1e-08, tol=5e-09 .....................
[CV]  C=0.01, degree=-0.9, gamma=1e-08, tol=5e-09, score=0.750, total=   0.0s
[CV] C=0.01, degree=-0.9, gamma=1e-08, tol=5e-09 .....................
[CV]  C=0.01, degree=-0.9, gamma=1e-08, tol=5e-09, score=1.000, total=   0.0s
[CV] C=0.01, degree=-0.9, gamma=1e-08, tol=5e-09 .....................
[CV]  C=0.01, degree=-0.9, gamma=1e-08, tol=5e-09, score=0.667, total=   0.0s


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = col

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.01], 'degree': [-0.9], 'gamma': [1e-08],
                         'tol': [5e-09]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=3)

In [53]:
# List best parameters and best score
print(gridModel.best_params_)
print(gridModel.best_score_)

{'C': 0.01, 'degree': -0.9, 'gamma': 1e-08, 'tol': 5e-09}
0.7833333333333333


In [54]:
# Testing the model
print(f"Training Data Score: {gridModel.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {gridModel.score(X_test_scaled, y_test)}")

Training Data Score: 0.7777777777777778
Testing Data Score: 0.8333333333333334


In [55]:
# Make predictions with the hypertuned model
predictions = gridModel.predict(X_test)
y_test_df = y_test.ravel()

prediction_df = pd.DataFrame({'predicted_values': predictions,
                   'Values_from_data': y_test_df})
prediction_df.head(60)

,predicted_values,Values_from_data
0,1,1
1,1,1
2,1,1
3,1,0
4,1,1
5,1,1


In [56]:
# SAVE MODEL
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'gridModel_model_v1.sav'
joblib.dump(gridModel, filename)

['gridModel_model_v1.sav']